In [2]:
import pandas as pd, numpy as np, os, math

In [3]:
outstanding_games = pd.read_csv("2020GamesLeft.csv")

In [4]:
allTheStats = pd.read_csv("../perfectlyCleaned_Data.csv")

In [7]:
allTheStats.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [8]:
outstanding_games.columns

Index(['Year', 'Week2', 'Day', 'Date', 'Time', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'TEAM_POINT_MARGIN', 'OPPONENTS_POINTS',
       'OPPONENTS_YARDS', 'OPPONENTS_TURNOVERS',
       'OPPONENTS_TURNOVER_DIFFERENTIAL', 'GAMEID', 'BINARY_WIN', 'Home_Team'],
      dtype='object')

## model1: OVERALL FPI (ESPN Team Eff Metric)

In [10]:
#model1: just OVERALL FPI (ESPN Team Eff Metric)
model_values = allTheStats.loc[:, ["TEAM_POINTS", "Power Index FPI"]]

In [11]:
from sklearn.model_selection import train_test_split
import sklearn.linear_model

In [15]:
model_1 = sklearn.linear_model.LinearRegression()

In [13]:
X = model_values.iloc[:,1].values
y = model_values.iloc[:,0].values

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [17]:
model_1.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [19]:
#R_sq Score
model_1.score(X_train, y_train)

0.11160052247107721

In [21]:
model_1.score(X_test, y_test)

0.12100326484896597

In [22]:
model_1.intercept_

array([22.64165239])

In [23]:
model_1.coef_

array([[0.64681882]])

In [25]:
y_pred = model_1.predict(X_test)

In [26]:
sq_resid = (y_pred - y_test)**2

In [27]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

7.449358973567071

In [30]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_1, params, verbose=3)

In [31]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.131, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.121, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.131, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.045, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.105, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.131, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [33]:
grid.best_score_

0.10659290608447188

In [34]:
ys = grid.predict(X_test)

In [35]:
resids = (ys - y_test)**2

In [36]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

7.449358973567071

In [37]:
from sklearn.metrics import r2_score

In [39]:
r2_score(y_test, ys)

0.12100326484896595

## model2:
### OVERALL FPI + TEAM OFFENSIVE FPI + THE OPPOSING_TEAM'S DEFENSIVE FPI + TEAM SPECIAL TEAMS FPI + Strength of Schedule FPI + WP Ranks

In [40]:
allTheStats.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [43]:
allTheStats2 = allTheStats.copy(deep=True)

In [44]:
#model2: All the ESPN EFF METRICS
#model_values = allTheStats.loc[:, ["TEAM_POINTS", "Power Index FPI"]]

In [50]:
opponentDef = allTheStats.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', "GAMEID"]]

In [78]:
model_values=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [80]:
model_values=model_values.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP']]

In [74]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2902 non-null   float64
 1   Power Index FPI      2902 non-null   float64
 2   Power Index OFF      2902 non-null   float64
 3   Power Index DEF_Opp  2902 non-null   float64
 4   Power Index ST       2902 non-null   float64
 5   Ranks SOS            2902 non-null   float64
 6   Ranks AVGWP          2902 non-null   float64
dtypes: float64(7)
memory usage: 181.4 KB


In [79]:
model_values.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', 'Power Index DEF_Opp']]

,FINAL_TEAM_NAME,Power Index DEF,Power Index DEF_Opp
0,Detroit Lions,-2.4,-2.7
1,Chicago Bears,-2.7,-2.4
2,Chicago Bears,-2.7,-2.4
3,Detroit Lions,-2.4,-2.7
4,Washington Redskins,-2.2,-2.7
...,...,...,...
2897,Kansas City Chiefs,1.0,-0.2
2898,Oakland Raiders,-3.4,1.3
2899,New England Patriots,1.3,-3.4
2900,Buffalo Bills,-0.2,-3.4


In [87]:
model_2 = sklearn.linear_model.LinearRegression(normalize=True)

In [88]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [91]:
model_2.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [92]:
#R_sq Score
model_2.score(X_train, y_train)

0.2117845417196058

In [93]:
model_2.score(X_test, y_test)

0.19001364187854686

In [94]:
model_2.intercept_

array([24.06222075])

In [95]:
model_2.coef_

array([[ 0.04133081,  0.67166743, -0.95707838,  0.47373876,  0.00707634,
        -0.09042084]])

In [96]:
y_pred = model_2.predict(X_test)

In [97]:
sq_resid = (y_pred - y_test)**2

In [98]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

6.950467849849554

In [99]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_2, params, verbose=3)

In [100]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.245, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.191, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.228, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.209, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.153, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.245, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [101]:
grid.best_score_

0.20522667173526582

In [102]:
ys = grid.predict(X_test)

In [103]:
resids = (ys - y_test)**2

In [104]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

6.950467849849554

In [105]:
from sklearn.metrics import r2_score

In [106]:
r2_score(y_test, ys)

0.19001364187854686

## Model 3:
### Past FPI and Introducing Yards and Turnovers

In [107]:
allTheStats2.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [108]:
opponentDef = allTheStats2.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', "GAMEID"]]

In [109]:
model_values=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [112]:
allTheStats3=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [110]:
model_values=model_values.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS"]]

In [111]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2902 non-null   float64
 1   Power Index FPI      2902 non-null   float64
 2   Power Index OFF      2902 non-null   float64
 3   Power Index DEF_Opp  2902 non-null   float64
 4   Power Index ST       2902 non-null   float64
 5   Ranks SOS            2902 non-null   float64
 6   Ranks AVGWP          2902 non-null   float64
 7   TEAM_YARDS           2902 non-null   float64
 8   TEAM_TURNOVERS       2902 non-null   float64
dtypes: float64(9)
memory usage: 226.7 KB


In [113]:
model_3 = sklearn.linear_model.LinearRegression(normalize=True)

In [114]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [120]:
model_3.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [121]:
#R_sq Score
model_3.score(X_train, y_train)

0.4887859670361537

In [122]:
model_3.score(X_test, y_test)

0.5273383363155932

In [123]:
model_3.intercept_

array([4.07569457])

In [124]:
model_3.coef_

array([[ 0.08343974,  0.11082988, -0.34514904,  0.36145503,  0.01845262,
        -0.08814872,  0.0644413 , -1.81601954]])

In [125]:
y_pred = model_3.predict(X_test)

In [126]:
sq_resid = (y_pred - y_test)**2

In [127]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

5.284216415845227

In [128]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_3, params, verbose=3)

In [129]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.401, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.495, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.477, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.508, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.513, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.401, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [130]:
grid.best_score_

0.4785656440179504

In [131]:
ys = grid.predict(X_test)

In [132]:
resids = (ys - y_test)**2

In [133]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

5.284216415845227

In [134]:
from sklearn.metrics import r2_score

In [135]:
r2_score(y_test, ys)

0.5273383363155932

## Model 4:
### Could A Classifier Work For a Regression Problem (Random Forest)

In [136]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [137]:
model_values=allTheStats3.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS"]]

In [138]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2902 non-null   float64
 1   Power Index FPI      2902 non-null   float64
 2   Power Index OFF      2902 non-null   float64
 3   Power Index DEF_Opp  2902 non-null   float64
 4   Power Index ST       2902 non-null   float64
 5   Ranks SOS            2902 non-null   float64
 6   Ranks AVGWP          2902 non-null   float64
 7   TEAM_YARDS           2902 non-null   float64
 8   TEAM_TURNOVERS       2902 non-null   float64
dtypes: float64(9)
memory usage: 226.7 KB


In [139]:
from sklearn.ensemble import RandomForestClassifier

In [140]:
rf = RandomForestClassifier(n_estimators=4000, criterion="entropy", max_depth=5)

In [141]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [142]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [143]:
rf.fit(X_train, y_train)

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=4000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [144]:
#R_sq Score
rf.score(X_train, y_train)

0.25505514705882354

In [145]:
rf.score(X_test, y_test)

0.08677685950413223

In [146]:
rf.classes_

array([ 0.,  3.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15.,
       16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
       29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41.,
       42., 43., 44., 45., 46., 47., 48., 49., 51., 52., 54., 55., 57.,
       59.])

In [152]:
y_pred = rf.predict(X_test)

In [153]:
sq_resid = (y_pred - y_test)**2

In [154]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

9.420603480333007

In [155]:
## STOP POINT FOR RF ##

In [162]:
from sklearn.model_selection import RandomizedSearchCV

In [163]:
# Grid Search
from sklearn.model_selection import RandomizedSearchCV
params = {'n_estimators':[4,40,400,4000,8000], 'criterion':["gini", "entropy"], "max_depth":[2, 5, 10]}
grid = RandomizedSearchCV(rf, params, verbose=3)

In [164]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=4000, max_depth=2, criterion=entropy ...............


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=4000, max_depth=2, criterion=entropy, score=0.078, total=  29.3s
[CV] n_estimators=4000, max_depth=2, criterion=entropy ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   29.2s remaining:    0.0s
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=4000, max_depth=2, criterion=entropy, score=0.078, total=  31.3s
[CV] n_estimators=4000, max_depth=2, criterion=entropy ...............


...

[CV]  n_estimators=8000, max_depth=10, criterion=gini, score=0.074, total=  24.5s
[CV] n_estimators=8000, max_depth=10, criterion=gini .................


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=8000, max_depth=10, criterion=gini, score=0.092, total=  24.3s


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  6.5min finished
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='entropy',
                                                    max_depth=5,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
            

In [165]:
grid.best_score_

0.09328271643994517

In [166]:
ys = grid.predict(X_test)

In [167]:
resids = (ys - y_test)**2

In [168]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

9.918588590639681

In [169]:
from sklearn.metrics import r2_score

In [170]:
r2_score(y_test, ys)

0.214525684460524

## Model 5:
### FPI, Yards, Turnovers + Expected Points Added per Play (EPA) and Success Rate for Offense overall and Pass EPA vs. Run EPA

In [171]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [173]:
model_values=allTheStats3.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS",  'EPA/play_RDBMS_Winner', 'OVR_SUCCESS_RATE_RDBMS_Winner', 'Dropback EPA_RDBMS_Winner',
                                  'DROPBACK_SR_RDBMS_Winner', 'Rush EPA_RDBMS_Winner', 'RUN_SR_RDBMS_Winner']]

In [174]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   TEAM_POINTS                    2902 non-null   float64
 1   Power Index FPI                2902 non-null   float64
 2   Power Index OFF                2902 non-null   float64
 3   Power Index DEF_Opp            2902 non-null   float64
 4   Power Index ST                 2902 non-null   float64
 5   Ranks SOS                      2902 non-null   float64
 6   Ranks AVGWP                    2902 non-null   float64
 7   TEAM_YARDS                     2902 non-null   float64
 8   TEAM_TURNOVERS                 2902 non-null   float64
 9   EPA/play_RDBMS_Winner          2902 non-null   float64
 10  OVR_SUCCESS_RATE_RDBMS_Winner  2902 non-null   float64
 11  Dropback EPA_RDBMS_Winner      2902 non-null   float64
 12  DROPBACK_SR_RDBMS_Winner       2902 non-null   f

In [175]:
model_5= sklearn.linear_model.LinearRegression(normalize=True)

In [176]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [177]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [178]:
model_5.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [179]:
#R_sq Score
model_5.score(X_train, y_train)

0.6302700899587667

In [180]:
model_5.score(X_test, y_test)

0.6277712268609107

In [181]:
model_5.intercept_

array([19.35315188])

In [182]:
model_5.coef_

array([[ 1.30057265e-01, -8.14878619e-02, -2.34768717e-02,
         8.55711147e-01,  2.11939999e-02, -5.30486944e-02,
         2.24907272e-02,  5.33776096e-01,  2.75554251e+01,
        -3.30534744e+01,  4.57205456e+00,  1.49555454e+01,
         2.51527195e+00,  7.17865874e+00]])

In [204]:
model_values

,TEAM_POINTS,Power Index FPI,Power Index OFF,Power Index DEF_Opp,Power Index ST,Ranks SOS,Ranks AVGWP,TEAM_YARDS,TEAM_TURNOVERS,EPA/play_RDBMS_Winner,OVR_SUCCESS_RATE_RDBMS_Winner,Dropback EPA_RDBMS_Winner,DROPBACK_SR_RDBMS_Winner,Rush EPA_RDBMS_Winner,RUN_SR_RDBMS_Winner
0,24.0,-2.0,0.9,-2.7,-0.5,15.0,13.0,349.0,0.0,0.180,0.468,0.257,0.500,-0.041,0.375
1,20.0,-2.6,0.1,-2.4,0.0,12.0,27.0,345.0,3.0,-0.008,0.442,-0.009,0.483,-0.007,0.391
2,34.0,-2.6,0.1,-2.4,0.0,12.0,27.0,444.0,1.0,0.053,0.436,0.227,0.543,-0.198,0.281
3,37.0,-2.0,0.9,-2.7,-0.5,15.0,13.0,546.0,3.0,0.151,0.476,0.286,0.491,-0.095,0.448
4,24.0,-2.8,-0.4,-2.7,-0.2,25.0,10.0,374.0,1.0,0.195,0.551,0.428,0.676,-0.075,0.406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2897,26.0,10.3,9.5,-0.2,-0.2,26.0,2.0,466.0,1.0,0.332,0.620,0.507,0.667,0.180,0.579
2898,20.0,-0.2,3.4,1.3,-0.1,1.0,20.0,375.0,3.0,-0.036,0.525,-0.001,0.564,-0.106,0.450
2899,36.0,-0.1,-1.8,-3.4,0.5,12.0,26.0,406.0,1.0,0.151,0.500,-0.057,0.469,0.346,0.529
2900,30.0,1.9,2.2,-3.4,-0.2,16.0,5.0,337.0,0.0,0.232,0.475,0.487,0.583,-0.169,0.304


In [183]:
y_pred = model_5.predict(X_test)

In [184]:
sq_resid = (y_pred - y_test)**2

In [185]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

4.647672409034977

In [186]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_3, params, verbose=3)

In [189]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.582, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.616, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.647, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.607, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.645, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.582, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [190]:
grid.best_score_

0.6193625275279359

In [191]:
ys = grid.predict(X_test)

In [192]:
resids = (ys - y_test)**2

In [193]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

4.647672409034978

In [194]:
from sklearn.metrics import r2_score

In [195]:
r2_score(y_test, ys)

0.6277712268609108

In [197]:
!cd ..

In [202]:
!ls

2020GamesLeft.csv
MachineLearningModels.ipynb


In [203]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [209]:
allTheStats3.groupby(["FINAL_TEAM_NAME"]).count()

,Unnamed: 0,Year,Week_x,Abbr_RDBMS_Winner,Week2,Day,Date,Time,Home_Team,Opponent,...,ESTIM. WINS_FO_ANNUAL_Winner,SCHEDULE RANK_FO_ANNUAL_Winner,VAR._FO_ANNUAL_Winner,TOTAL_GAME_POINTS,TOTAL_GAME_YARDS,TOTAL_GAME_TURNOVERS,PLAYOFFS_BINARY,GAME_TYPE,FINAL_TEAM_NAME_Opp,Power Index DEF_Opp
FINAL_TEAM_NAME,,,,,,,,,,,,,,,,,,,,,
Arizona Cardinals,90,90,90,90,90,90,90,90,90,90,...,90,90,90,90,90,90,90,90,90,90
Atlanta Falcons,94,94,94,94,94,94,94,94,94,94,...,94,94,94,94,94,94,94,94,94,94
Baltimore Ravens,90,90,90,90,90,90,90,90,90,90,...,90,90,90,90,90,90,90,90,90,90
Buffalo Bills,90,90,90,90,90,90,90,90,90,90,...,90,90,90,90,90,90,90,90,90,90
Carolina Panthers,92,92,92,92,92,92,92,92,92,92,...,92,92,92,92,92,92,92,92,92,92
Chicago Bears,88,88,88,88,88,88,88,88,88,88,...,88,88,88,88,88,88,88,88,88,88
Cincinnati Bengals,88,88,88,88,88,88,88,88,88,88,...,88,88,88,88,88,88,88,88,88,88
Cleveland Browns,88,88,88,88,88,88,88,88,88,88,...,88,88,88,88,88,88,88,88,88,88
Dallas Cowboys,92,92,92,92,92,92,92,92,92,92,...,92,92,92,92,92,92,92,92,92,92


In [211]:
elo = pd.read_csv("../Advanced Stats/538.csv")

In [217]:
fpi = pd.read_csv("../2015-2020_FPI.csv")

In [220]:
fpi=fpi.dropna(how="all")

In [219]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [223]:
#fpi["Ranks REM SOS"]
#fpi["Power Index TREND"]
def cleanme(c):
    if c == "--":
        return 0
    else:
        return int(c)

In [227]:
fpi["Ranks REM SOS"]=fpi["Ranks REM SOS"].apply(cleanme)

In [229]:
fpi["Power Index TREND"]=fpi["Power Index TREND"].apply(cleanme)

In [230]:
epa = pd.read_csv("../Advanced Stats/epa2015_2020.csv")

In [231]:
epa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2936 entries, 0 to 2935
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TEAM              2936 non-null   object 
 1   Abbr              2936 non-null   object 
 2   EPA/play          2936 non-null   float64
 3   OVR_SUCCESS_RATE  2936 non-null   float64
 4   Dropback EPA      2936 non-null   float64
 5   DROPBACK_SR       2936 non-null   float64
 6   Rush EPA          2936 non-null   float64
 7   RUN_SR            2936 non-null   float64
 8   week              2936 non-null   int64  
 9   year              2936 non-null   int64  
dtypes: float64(6), int64(2), object(2)
memory usage: 229.5+ KB


In [232]:
DVOA_Y = pd.read_csv("../Advanced Stats/DVOA_yearly.csv")

In [233]:
DVOA_W = pd.read_csv("../Advanced Stats/DVOA_weekly.csv")

In [237]:
DVOA_W=DVOA_W.iloc[:, :16]

In [239]:
DVOA_Y.columns

Index(['RK', 'Team', 'Abbr', 'TOTAL DVOA', 'LAST WEEK', 'DAVE_WEI_DVOA',
       'RANK', 'W-L', 'OFFENSE DVOA', 'OFF. RANK', 'DEFENSE DVOA', 'DEF. RANK',
       'S.T. DVOA', 'S.T. RANK', 'WEEK', 'YEAR', 'WEIGHTED DVOA', 'RANK.1',
       'non-adj VOA', 'Last Year', 'ESTIM. WINS', 'EW RANK', 'SCHEDULE RANK',
       'RANK.2', 'PYTH WINS', 'RANK.3', 'VAR.', 'RANK.4'],
      dtype='object')

In [240]:
ycols = ['RK', 'Team', 'Abbr', 'TOTAL DVOA', 'DAVE_WEI_DVOA',
       'RANK', 'W-L', 'OFFENSE DVOA', 'OFF. RANK', 'DEFENSE DVOA', 'DEF. RANK',
       'S.T. DVOA', 'S.T. RANK', 'WEEK', 'YEAR', 'WEIGHTED DVOA', 'RANK.1',
       'non-adj VOA', 'Last Year', 'ESTIM. WINS', 'EW RANK', 'VAR.']

In [242]:
DVOA_Y = DVOA_Y.loc[:, ycols]

In [251]:
DVOA_Y.columns

Index(['RK', 'Team', 'Abbr', 'TOTAL DVOA', 'LAST WEEK', 'DAVE_WEI_DVOA',
       'RANK', 'W-L', 'OFFENSE DVOA', 'OFF. RANK', 'DEFENSE DVOA', 'DEF. RANK',
       'S.T. DVOA', 'S.T. RANK', 'WEEK', 'YEAR', 'WEIGHTED DVOA', 'RANK.1',
       'non-adj VOA', 'Last Year', 'ESTIM. WINS', 'EW RANK', 'SCHEDULE RANK',
       'RANK.2', 'PYTH WINS', 'RANK.3', 'VAR.', 'RANK.4'],
      dtype='object')

In [260]:
fpi.columns

Index(['TEAM', 'Year ', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power Index OFF', 'Power Index DEF',
       'Power Index ST', 'Ranks SOS', 'Ranks REM SOS', 'Ranks AVGWP'],
      dtype='object')

In [256]:
elo.columns

Index(['ELO WITH QB', 'ELO CHANGE', 'CURRENT QB ADJ', 'TEAM', 'DIVISION',
       'PROJ WINS', 'PROJ LOSES', 'POINT DIFF', 'MAKE PLAYOFFS',
       'WIN DIVISION', '1st-Round Bye', 'Win Superbowl', 'Before Week',
       'Year'],
      dtype='object')

In [254]:
DVOA_W.columns

Index(['RK', 'Team', 'Abbr', 'TOTAL DVOA', 'LAST WEEK', 'DAVE_WEI_DVOA',
       'RANK', 'W-L', 'OFFENSE DVOA', 'OFF. RANK', 'DEFENSE DVOA', 'DEF. RANK',
       'S.T. DVOA', 'S.T. RANK', 'WEEK', 'YEAR'],
      dtype='object')

In [277]:
remain = pd.read_csv("leftovers.csv")

In [262]:
remain.columns

Index(['Year', 'Week', 'Day', 'Date', 'Time', 'FINAL_TEAM_NAME', 'TEAM_POINTS',
       'TEAM_YARDS', 'TEAM_TURNOVERS', 'TEAM_TODiff', 'GAMEID', 'Opponent',
       'Opponent_PTS', 'Opponent_Yds', 'Opponents_TO', 'Opponent_TODiff',
       'HOME_TEAM', 'BINARY_WIN'],
      dtype='object')

In [278]:
remain2 = pd.merge(remain, DVOA_Y, left_on=["FINAL_TEAM_NAME", "Year"], right_on=["Team", "YEAR"])

In [279]:
remain3 = pd.merge(remain2, DVOA_W, left_on=["FINAL_TEAM_NAME", "Year", "Week"], right_on=["Team", "YEAR", "WEEK"])

In [265]:
#remain4 = pd.merge(remain3, elo, left_on=["FINAL_TEAM_NAME", "Year", "Week"], right_on=["TEAM", "Year", "Before Week"])

In [280]:
remain5 = pd.merge(remain3, fpi, left_on=["FINAL_TEAM_NAME", "Year"], right_on=["TEAM", "Year "])

In [281]:
remain5.shape

(32, 74)

In [275]:
remain5.to_csv("remain22.csv", index=False)

In [276]:
remain5.columns

Index(['Year', 'Week', 'Day', 'Date', 'Time', 'FINAL_TEAM_NAME', 'TEAM_POINTS',
       'TEAM_YARDS', 'TEAM_TURNOVERS', 'TEAM_TODiff', 'GAMEID', 'Opponent',
       'Opponent_PTS', 'Opponent_Yds', 'Opponents_TO', 'Opponent_TODiff',
       'HOME_TEAM', 'BINARY_WIN', 'RK_x', 'Team_x', 'Abbr_x', 'TOTAL DVOA_x',
       'LAST WEEK_x', 'DAVE_WEI_DVOA_x', 'RANK_x', 'W-L_x', 'OFFENSE DVOA_x',
       'OFF. RANK_x', 'DEFENSE DVOA_x', 'DEF. RANK_x', 'S.T. DVOA_x',
       'S.T. RANK_x', 'WEEK_x', 'YEAR_x', 'WEIGHTED DVOA', 'RANK.1',
       'non-adj VOA', 'Last Year', 'ESTIM. WINS', 'EW RANK', 'SCHEDULE RANK',
       'RANK.2', 'PYTH WINS', 'RANK.3', 'VAR.', 'RANK.4', 'RK_y', 'Team_y',
       'Abbr_y', 'TOTAL DVOA_y', 'LAST WEEK_y', 'DAVE_WEI_DVOA_y', 'RANK_y',
       'W-L_y', 'OFFENSE DVOA_y', 'OFF. RANK_y', 'DEFENSE DVOA_y',
       'DEF. RANK_y', 'S.T. DVOA_y', 'S.T. RANK_y', 'WEEK_y', 'YEAR_y', 'TEAM',
       'Year ', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power 

In [3]:
final_df3 = pd.read_csv("perfectlyCleanedData2.csv")

In [13]:
ty = final_df3.iloc[0, 21]

In [11]:
import re

In [21]:
regex = re.compile(".+/.+/")

In [39]:
regex.search("7/2/2000")#[0][:-1].replace("/", "-")

<re.Match object; span=(0, 4), match='7/2/'>

In [42]:
def cleanDate(d):
    match = regex.search(d)
    if match != None:
        match = match[0][:-1]
        match = match.replace("/", "-")
        return match
    else:
        return d

In [44]:
final_df3["W-L-T"]=final_df3["W-L-T"].apply(cleanDate)

In [46]:
final_df3.to_csv("perfectlyCleanedData3.csv", index=False)

## Redux

In [1]:
import pandas as pd, numpy as np, os, math

In [3]:
allTheStats = pd.read_csv("perfectlyCleanedData3.csv")

In [4]:
allTheStats.columns

Index(['Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day', 'Date', 'Time',
       'Home_Team', 'FINAL_TEAM_NAME', 'Opponent', 'TEAM_POINTS', 'TEAM_YARDS',
       'TEAM_TURNOVERS', 'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS',
       'OPPONENTS_YARDS', 'OPPONENTS_TURNOVERS',
       'OPPONENTS_TURNOVER_DIFFERENTIAL', 'TEAM_POINT_MARGIN', 'GAMEID',
       'BINARY_WIN', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power Index OFF', 'Power Index DEF',
       'Power Index ST', 'Ranks SOS', 'Ranks REM SOS', 'Ranks AVGWP',
       'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner',
       'OVR_

## model1: OVERALL FPI (ESPN Team Eff Metric)

In [5]:
#model1: just OVERALL FPI (ESPN Team Eff Metric)
model_values = allTheStats.loc[:, ["TEAM_POINTS", "Power Index FPI"]]

In [6]:
from sklearn.model_selection import train_test_split
import sklearn.linear_model

In [7]:
model_1 = sklearn.linear_model.LinearRegression()

In [8]:
X = model_values.iloc[:,1].values
y = model_values.iloc[:,0].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [10]:
model_1.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [11]:
#R_sq Score
model_1.score(X_train, y_train)

0.12837775897102

In [12]:
model_1.score(X_test, y_test)

0.07136027439629333

In [13]:
model_1.intercept_

array([22.78005983])

In [14]:
model_1.coef_

array([[0.69949756]])

In [15]:
y_pred = model_1.predict(X_test)

In [16]:
sq_resid = (y_pred - y_test)**2

In [17]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

7.321622897853451

In [18]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_1, params, verbose=3)

In [19]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.105, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.116, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.138, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.137, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.135, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.105, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [20]:
grid.best_score_

0.12621251574894304

In [21]:
ys = grid.predict(X_test)

In [22]:
resids = (ys - y_test)**2

In [23]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

7.321622897853449

In [24]:
from sklearn.metrics import r2_score

In [25]:
r2_score(y_test, ys)

0.07136027439629322

## model2:
### OVERALL FPI + TEAM OFFENSIVE FPI + THE OPPOSING_TEAM'S DEFENSIVE FPI + TEAM SPECIAL TEAMS FPI + Strength of Schedule FPI + WP Ranks

In [26]:
allTheStats.columns

Index(['Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day', 'Date', 'Time',
       'Home_Team', 'FINAL_TEAM_NAME', 'Opponent', 'TEAM_POINTS', 'TEAM_YARDS',
       'TEAM_TURNOVERS', 'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS',
       'OPPONENTS_YARDS', 'OPPONENTS_TURNOVERS',
       'OPPONENTS_TURNOVER_DIFFERENTIAL', 'TEAM_POINT_MARGIN', 'GAMEID',
       'BINARY_WIN', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power Index OFF', 'Power Index DEF',
       'Power Index ST', 'Ranks SOS', 'Ranks REM SOS', 'Ranks AVGWP',
       'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner',
       'OVR_

In [27]:
allTheStats2 = allTheStats.copy(deep=True)

In [28]:
#model2: All the ESPN EFF METRICS
#model_values = allTheStats.loc[:, ["TEAM_POINTS", "Power Index FPI"]]

In [29]:
opponentDef = allTheStats.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', "GAMEID"]]

In [30]:
model_values=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [31]:
model_values=model_values.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP']]

In [32]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2932 entries, 0 to 2931
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2932 non-null   int64  
 1   Power Index FPI      2932 non-null   float64
 2   Power Index OFF      2932 non-null   float64
 3   Power Index DEF_Opp  2932 non-null   float64
 4   Power Index ST       2932 non-null   float64
 5   Ranks SOS            2932 non-null   int64  
 6   Ranks AVGWP          2932 non-null   int64  
dtypes: float64(4), int64(3)
memory usage: 183.2 KB


In [35]:
model_values.loc[:, ["TEAM_POINTS",'Power Index OFF', 'Power Index DEF_Opp']]

,TEAM_POINTS,Power Index OFF,Power Index DEF_Opp
0,24,0.9,-2.7
1,20,0.1,-2.4
2,34,0.1,-2.4
3,37,0.9,-2.7
4,24,-0.4,-2.7
...,...,...,...
2927,31,2.4,-2.2
2928,42,2.2,-2.2
2929,42,1.6,-2.2
2930,38,6.4,5.1


In [36]:
model_2 = sklearn.linear_model.LinearRegression(normalize=True)

In [37]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [39]:
model_2.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [40]:
#R_sq Score
model_2.score(X_train, y_train)

0.20154513906522328

In [41]:
model_2.score(X_test, y_test)

0.21883076204876128

In [42]:
model_2.intercept_

array([24.05059979])

In [43]:
model_2.coef_

array([[ 0.09374783,  0.59316021, -0.92479362, -0.04941855,  0.03119231,
        -0.09964679]])

In [44]:
y_pred = model_2.predict(X_test)

In [45]:
sq_resid = (y_pred - y_test)**2

In [46]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

7.008383989531295

In [47]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_2, params, verbose=3)

In [48]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.140, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.195, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.195, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.275, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.169, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.140, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [49]:
grid.best_score_

0.19516793987880796

In [50]:
ys = grid.predict(X_test)

In [51]:
resids = (ys - y_test)**2

In [52]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

7.008383989531295

In [53]:
from sklearn.metrics import r2_score

In [54]:
r2_score(y_test, ys)

0.21883076204876128

## Model 3:
### Past FPI and Introducing Yards and Turnovers

In [55]:
allTheStats2.columns

Index(['Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day', 'Date', 'Time',
       'Home_Team', 'FINAL_TEAM_NAME', 'Opponent', 'TEAM_POINTS', 'TEAM_YARDS',
       'TEAM_TURNOVERS', 'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS',
       'OPPONENTS_YARDS', 'OPPONENTS_TURNOVERS',
       'OPPONENTS_TURNOVER_DIFFERENTIAL', 'TEAM_POINT_MARGIN', 'GAMEID',
       'BINARY_WIN', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power Index OFF', 'Power Index DEF',
       'Power Index ST', 'Ranks SOS', 'Ranks REM SOS', 'Ranks AVGWP',
       'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner',
       'OVR_

In [56]:
opponentDef = allTheStats2.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', "GAMEID"]]

In [57]:
model_values=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [58]:
allTheStats3=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [59]:
model_values=model_values.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS"]]

In [60]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2932 entries, 0 to 2931
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2932 non-null   int64  
 1   Power Index FPI      2932 non-null   float64
 2   Power Index OFF      2932 non-null   float64
 3   Power Index DEF_Opp  2932 non-null   float64
 4   Power Index ST       2932 non-null   float64
 5   Ranks SOS            2932 non-null   int64  
 6   Ranks AVGWP          2932 non-null   int64  
 7   TEAM_YARDS           2932 non-null   int64  
 8   TEAM_TURNOVERS       2932 non-null   int64  
dtypes: float64(4), int64(5)
memory usage: 229.1 KB


In [61]:
model_3 = sklearn.linear_model.LinearRegression(normalize=True)

In [62]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [64]:
model_3.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [65]:
#R_sq Score
model_3.score(X_train, y_train)

0.4844359762627359

In [66]:
model_3.score(X_test, y_test)

0.5343930244400317

In [67]:
model_3.intercept_

array([4.46879657])

In [68]:
model_3.coef_

array([[ 0.09629109,  0.07018035, -0.25159812,  0.17847358,  0.00694826,
        -0.10720859,  0.06490841, -1.87130443]])

In [69]:
y_pred = model_3.predict(X_test)

In [70]:
sq_resid = (y_pred - y_test)**2

In [71]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

5.36006083235951

In [72]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_3, params, verbose=3)

In [73]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.541, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.488, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.445, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.479, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.440, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.541, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [74]:
grid.best_score_

0.47870269881686944

In [75]:
ys = grid.predict(X_test)

In [76]:
resids = (ys - y_test)**2

In [77]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

5.36006083235951

In [78]:
from sklearn.metrics import r2_score

In [79]:
r2_score(y_test, ys)

0.5343930244400317

## Model 4:
### Could A Classifier Work For a Regression Problem (Random Forest)

In [80]:
allTheStats3.columns

Index(['Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day', 'Date', 'Time',
       'Home_Team', 'FINAL_TEAM_NAME', 'Opponent', 'TEAM_POINTS', 'TEAM_YARDS',
       'TEAM_TURNOVERS', 'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS',
       'OPPONENTS_YARDS', 'OPPONENTS_TURNOVERS',
       'OPPONENTS_TURNOVER_DIFFERENTIAL', 'TEAM_POINT_MARGIN', 'GAMEID',
       'BINARY_WIN', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power Index OFF', 'Power Index DEF',
       'Power Index ST', 'Ranks SOS', 'Ranks REM SOS', 'Ranks AVGWP',
       'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner',
       'OVR_

In [81]:
model_values=allTheStats3.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS"]]

In [82]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2932 entries, 0 to 2931
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2932 non-null   int64  
 1   Power Index FPI      2932 non-null   float64
 2   Power Index OFF      2932 non-null   float64
 3   Power Index DEF_Opp  2932 non-null   float64
 4   Power Index ST       2932 non-null   float64
 5   Ranks SOS            2932 non-null   int64  
 6   Ranks AVGWP          2932 non-null   int64  
 7   TEAM_YARDS           2932 non-null   int64  
 8   TEAM_TURNOVERS       2932 non-null   int64  
dtypes: float64(4), int64(5)
memory usage: 229.1 KB


In [83]:
from sklearn.ensemble import RandomForestClassifier

In [84]:
rf = RandomForestClassifier(n_estimators=4000, criterion="entropy", max_depth=5)

In [85]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [87]:
rf.fit(X_train, y_train)

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=4000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [88]:
#R_sq Score
rf.score(X_train, y_train)

0.24010914051841747

In [89]:
rf.score(X_test, y_test)

0.07503410641200546

In [90]:
rf.classes_

array([ 0,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 51, 52, 54, 55,
       57], dtype=int64)

In [91]:
y_pred = rf.predict(X_test)

In [92]:
sq_resid = (y_pred - y_test)**2

In [93]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

9.319375606051864

In [94]:
## STOP POINT FOR RF ##

In [95]:
from sklearn.model_selection import RandomizedSearchCV

In [96]:
# Grid Search
from sklearn.model_selection import RandomizedSearchCV
params = {'n_estimators':[4,40,400,4000,8000], 'criterion':["gini", "entropy"], "max_depth":[2, 5, 10]}
grid = RandomizedSearchCV(rf, params, verbose=3)

In [97]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=40, max_depth=10, criterion=entropy ................


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=40, max_depth=10, criterion=entropy, score=0.061, total=   0.2s
[CV] n_estimators=40, max_depth=10, criterion=entropy ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=40, max_depth=10, criterion=entropy, score=0.075, total=   0.2s
[CV] n_estimators=40, max_depth=10, criterion=entropy ................


...

[CV]  n_estimators=8000, max_depth=2, criterion=entropy, score=0.109, total=  14.3s
[CV] n_estimators=8000, max_depth=2, criterion=entropy ...............


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=8000, max_depth=2, criterion=entropy, score=0.082, total=  14.2s
[CV] n_estimators=4, max_depth=5, criterion=gini .....................
[CV]  n_estimators=4, max_depth=5, criterion=gini, score=0.070, total=   0.0s
[CV] n_estimators=4, max_depth=5, criterion=gini .....................
[CV]  n_estimators=4, max_depth=5, criterion=gini, score=0.077, total=   0.0s
[CV] n_estimators=4, max_depth=5, criterion=gini .....................
[CV]  n_estimators=4, max_depth=5, criterion=gini, score=0.091, total=   0.0s
[CV] n_estimators=4, max_depth=5, criterion=gini .....................
[CV]  n_estimators=4, max_depth=5, criterion=gini, score=0.114, total=   0.0s
[CV] n_estimators=4, max_depth=5, criterion=gini .....................
[CV]  n_estimators=4, max_depth=5, criterion=gini, score=0.075, total=   0.0s


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was pa

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='entropy',
                                                    max_depth=5,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
            

In [98]:
grid.best_score_

0.09594533029612755

In [166]:
ys = grid.predict(X_test)

In [167]:
resids = (ys - y_test)**2

In [168]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

9.918588590639681

In [169]:
from sklearn.metrics import r2_score

In [170]:
r2_score(y_test, ys)

0.214525684460524

In [99]:
rf2 = RandomForestClassifier(n_estimators=40, criterion="gini", max_depth=5)

In [100]:
rf2.fit(X_train, y_train)

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [101]:
rf2.score(X_train, y_train)

0.24874943155979992

In [102]:
rf2.score(X_test, y_test)

0.07776261937244201

In [103]:
y_pred = rf2.predict(X_test)

In [104]:
sq_resid = (y_pred - y_test)**2

In [105]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

9.431611292991295

## Model 5:
### FPI, Yards, Turnovers + Expected Points Added per Play (EPA) and Success Rate for Offense overall and Pass EPA vs. Run EPA

In [106]:
allTheStats3.columns

Index(['Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day', 'Date', 'Time',
       'Home_Team', 'FINAL_TEAM_NAME', 'Opponent', 'TEAM_POINTS', 'TEAM_YARDS',
       'TEAM_TURNOVERS', 'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS',
       'OPPONENTS_YARDS', 'OPPONENTS_TURNOVERS',
       'OPPONENTS_TURNOVER_DIFFERENTIAL', 'TEAM_POINT_MARGIN', 'GAMEID',
       'BINARY_WIN', 'W-L-T', 'Power Index FPI', 'Power Index RK',
       'Power Index TREND', 'Power Index OFF', 'Power Index DEF',
       'Power Index ST', 'Ranks SOS', 'Ranks REM SOS', 'Ranks AVGWP',
       'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner',
       'OVR_

In [107]:
model_values=allTheStats3.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS",  'EPA/play_RDBMS_Winner', 'OVR_SUCCESS_RATE_RDBMS_Winner', 'Dropback EPA_RDBMS_Winner',
                                  'DROPBACK_SR_RDBMS_Winner', 'Rush EPA_RDBMS_Winner', 'RUN_SR_RDBMS_Winner']]

In [108]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2932 entries, 0 to 2931
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   TEAM_POINTS                    2932 non-null   int64  
 1   Power Index FPI                2932 non-null   float64
 2   Power Index OFF                2932 non-null   float64
 3   Power Index DEF_Opp            2932 non-null   float64
 4   Power Index ST                 2932 non-null   float64
 5   Ranks SOS                      2932 non-null   int64  
 6   Ranks AVGWP                    2932 non-null   int64  
 7   TEAM_YARDS                     2932 non-null   int64  
 8   TEAM_TURNOVERS                 2932 non-null   int64  
 9   EPA/play_RDBMS_Winner          2932 non-null   float64
 10  OVR_SUCCESS_RATE_RDBMS_Winner  2932 non-null   float64
 11  Dropback EPA_RDBMS_Winner      2932 non-null   float64
 12  DROPBACK_SR_RDBMS_Winner       2932 non-null   f

In [109]:
model_5= sklearn.linear_model.LinearRegression(normalize=True)

In [110]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [112]:
model_5.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [113]:
#R_sq Score
model_5.score(X_train, y_train)

0.6248639025345639

In [114]:
model_5.score(X_test, y_test)

0.612299807470251

In [115]:
model_5.intercept_

array([19.25145464])

In [116]:
model_5.coef_

array([[ 1.37686281e-01, -7.08405747e-02, -6.34908984e-02,
         6.95595163e-01,  3.26944863e-02, -4.09261346e-02,
         2.15664307e-02,  5.02436037e-01,  2.74083846e+01,
        -2.43643998e+01,  4.94395420e+00,  9.57538835e+00,
         2.60905471e+00,  4.11135591e+00]])

In [117]:
model_values

,TEAM_POINTS,Power Index FPI,Power Index OFF,Power Index DEF_Opp,Power Index ST,Ranks SOS,Ranks AVGWP,TEAM_YARDS,TEAM_TURNOVERS,EPA/play_RDBMS_Winner,OVR_SUCCESS_RATE_RDBMS_Winner,Dropback EPA_RDBMS_Winner,DROPBACK_SR_RDBMS_Winner,Rush EPA_RDBMS_Winner,RUN_SR_RDBMS_Winner
0,24,-2.0,0.9,-2.7,-0.5,15,13,349,0,0.180,0.468,0.257,0.500,-0.041,0.375
1,20,-2.6,0.1,-2.4,0.0,12,27,345,3,-0.008,0.442,-0.009,0.483,-0.007,0.391
2,34,-2.6,0.1,-2.4,0.0,12,27,444,1,0.053,0.436,0.227,0.543,-0.198,0.281
3,37,-2.0,0.9,-2.7,-0.5,15,13,546,3,0.151,0.476,0.286,0.491,-0.095,0.448
4,24,-2.8,-0.4,-2.7,-0.2,25,10,374,1,0.195,0.551,0.428,0.676,-0.075,0.406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2927,31,1.9,2.4,-2.2,-0.5,9,17,444,1,0.057,0.453,0.204,0.484,-0.081,0.424
2928,42,1.9,2.2,-2.2,-0.2,16,5,334,0,0.030,0.514,0.018,0.560,0.060,0.400
2929,42,-2.4,1.6,-2.2,0.2,7,27,607,2,0.299,0.478,0.567,0.581,-0.144,0.308
2930,38,4.7,6.4,5.1,-0.4,2,3,324,0,-0.314,0.339,-0.366,0.357,-0.205,0.300


In [118]:
y_pred = model_5.predict(X_test)

In [119]:
sq_resid = (y_pred - y_test)**2

In [120]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

4.7839731537685095

In [123]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_5, params, verbose=3)

In [124]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.634, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.625, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.586, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.615, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.634, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.634, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [125]:
grid.best_score_

0.6188779739484955

In [126]:
ys = grid.predict(X_test)

In [127]:
resids = (ys - y_test)**2

In [128]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

4.7839731537685095

In [129]:
from sklearn.metrics import r2_score

In [130]:
r2_score(y_test, ys)

0.612299807470251

In [203]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'